In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models
import os

In [3]:
resnet_model = tf.keras.models.load_model('Resnet_18_model.h5')

In [4]:
# Converting the resnet18 tensorflow model to tensorflow lite model
converter = tf.lite.TFLiteConverter.from_keras_model(resnet_model)
Resnet_18_tflite = converter.convert()
open('Resnet_18_model.tflite','wb').write(Resnet_18_tflite)

INFO:tensorflow:Assets written to: C:\Users\ATHARV~1\AppData\Local\Temp\tmpk7h8bgc9\assets


INFO:tensorflow:Assets written to: C:\Users\ATHARV~1\AppData\Local\Temp\tmpk7h8bgc9\assets


Saved artifact at 'C:\Users\ATHARV~1\AppData\Local\Temp\tmpk7h8bgc9'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  2681295235280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2681295236048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2681295237392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2681295237968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2681295237008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2681295237200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2681295239120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2681295235856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2681295234128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2681295233552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  26

44723856

In [16]:
h5_size = os.path.getsize('Resnet_18_model.h5')
tflite_size = os.path.getsize('Resnet_18_model.tflite')
MB = tflite_size/(1024*1024)

MB

42.65199279785156

In [25]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

class_cifer10 = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]
# Load and preprocess a random image
def preprocess_image(image_path, target_size):
    image = load_img(image_path, target_size=target_size)
    image = img_to_array(image)
    image = image / 255.0  # Normalize to [0, 1]
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

# Test the H5 model
def test_h5_model(h5_model_path, image_path):
    # Load the model
    model = tf.keras.models.load_model(h5_model_path)
    
    # Preprocess the image
    input_size = (32, 32)  # CIFAR-10 image size
    image = preprocess_image(image_path, target_size=input_size)

    # Make prediction
    class_cifer10 = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]
    predictions = model.predict(image)
    predicted_class = np.argmax(predictions)
    print(f"H5 Model Prediction: {predicted_class} \nclass: {class_cifer10[predicted_class]}")

# Test the TFLite model
def test_tflite_model(tflite_model_path, image_path):
    # Load the TFLite model
    interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
    interpreter.allocate_tensors()

    # Get input and output details
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Preprocess the image
    input_size = tuple(input_details[0]['shape'][1:3])  # Extract input size from model
    image = preprocess_image(image_path, target_size=input_size)

    # Set the tensor
    interpreter.set_tensor(input_details[0]['index'], image)

    # Run inference
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    
    
    predicted_class = np.argmax(output_data)
    print(f"TFLite Model Prediction: {predicted_class} \nPredicted class: {class_cifer10[predicted_class]}\n {output_data}")

if __name__ == "__main__":
    # Paths to models and test image
    h5_model_path = "Resnet_18_model.h5"
    tflite_model_path = "Resnet_18_model.tflite"
    test_image_path = r"C:\Users\Atharva Darke\Desktop\pexels-photo-6696645.jpeg"  # Replace with the path to your test image

    # Test H5 and TFLite models
    test_h5_model( h5_model_path , test_image_path )
    test_tflite_model(tflite_model_path, test_image_path)




1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 849ms/step
H5 Model Prediction: 7 
class: horse
TFLite Model Prediction: 7 
Predicted class: horse
 [[5.2472547e-08 1.4291823e-09 1.5779995e-07 1.3531459e-07 4.9616285e-02
  7.2096562e-04 2.7177251e-07 9.4966203e-01 3.8483533e-10 3.6966259e-08]]
